## Import necessary modules

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### Import the data

In [52]:
bom = pd.read_csv("data/bom.movie_gross.csv.gz")
imdb_names = pd.read_csv("data/imdb.name.basics.csv.gz")
imdb_title_akas = pd.read_csv("data/imdb.title.akas.csv.gz")
imdb_title_basics = pd.read_csv("data/imdb.title.basics.csv.gz")
imdb_title_princ = pd.read_csv("data/imdb.title.principals.csv.gz")
imdb_title_ratings = pd.read_csv("data/imdb.title.ratings.csv.gz")
tmdb = pd.read_csv("data/tmdb.movies.csv.gz")
rt_movie_info = pd.read_csv('data/rt.movie_info.tsv.gz', sep='\t')
rt_reviews = pd.read_csv("data/rt.reviews.tsv.gz", sep='\t', encoding='latin-1')
tn_movie_budgets = pd.read_csv("data/tn.movie_budgets.csv.gz", sep=',')

In [41]:
inner_bom_tmdb = bom.merge(tmdb, left_on='title', right_on='title')

In [45]:
imdb_title_basics = pd.read_csv("data/imdb.title.basics.csv.gz")

In [55]:
imdb_names[imdb_names['primary_name'] == 'Brad Pitt']

,nconst,primary_name,birth_year,death_year,primary_profession,known_for_titles
24026,nm0000093,Brad Pitt,1963.0,NaN,"actor,producer,soundtrack","tt1210166,tt0114746,tt0356910,tt0816711"
